In [1]:
from importlib import reload
import platform, os, sys, datetime, re
import multiprocessing
from os.path import join
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# sys.path.append(join(root_dir,'cvtracer'))
from cvt.TrAQ.Tank import Tank
from cvt.TrAQ.Tracker import Tracker
from cvt.utils import *
from converter import Converter


tank_diameter_vs_age = { 7:9.6, 14:10.4, 21:12.8, 28:17.7, 42:33.8 }


default_settings = dict(
    t_start        = 0,     # Time at which to start tracking, in seconds.
    t_end          = -1,    # Time at which to end tracking, in seconds.
    
    # Contour detection.
    n_pixel_blur   =  7,    # square-root of n-pixels for threshold blurring
    block_size     = 15,    # contour block size
    thresh_offset  = 13,    # threshold offset for contour-finding
    min_area       = 30,    # minimum area for detection
    max_area       = 400,   # maximum area for detection
    ideal_area     = 50,    # ideal area to rank contours in first frame (default=(min_area+max_area)/2)
    max_aspect     = 20,    # maximum aspect ratio for detection
    ideal_aspect   = 3,     # ideal aspect ratio to rank contours in first frame (default=max_aspect/2)
    area_penalty   = 0.5,   # weight of area change when connecting fish across frames
    reversal_threshold = 0.5, # average frame-to-frame displacement against the director 
                            # over the last few frames to trigger a reversal. 
    RGB            = True,  # track in color, false does greyscale
    online_viewer  = False, # Toggle live preview of tracking.

    # Background subtraction (for naive background subtraction only).
    bkgSub_options = dict( n_training_frames = 100, # number of frames used to compute background
                           t_start = 0, t_end = -1, # time range used to compute background
                           contrast_factor = 4 ),   # post-subtraction contrast enhancement factor
    
    # What information to draw on the tracking output video.
    video_output_options = dict( tank=True, points=False, directors=True, timestamp=True, 
                                 contours=True, contour_color=(100,255,0), contour_thickness=1 )
    )


def create_settings_(input_file, tracking_dir, settings):
    settings = settings.copy()
    globals().update(settings)
    
    ''' Extract trial info from the filename. '''
    filename,ext = os.path.splitext(os.path.basename(input_file))
    pop,_,age,group,Nfish = filename.split('_')[:5]
    Nfish    = int(re.findall('\d+',Nfish)[0])
    age      = int(age[:-3])

    ''' Define and create necessary folders/files/links. '''
    output_dir = join(tracking_dir,filename)
    new_input_file = input_file
    if not 'windows' in platform.system().lower():
        new_input_file = join(output_dir,'raw'+ext)
    trial_file    = join(output_dir,'trial.pik')
    tank_file     = join(output_dir,'tank.pik')
    log_file      = join(output_dir,'log.txt')
    
    # Return as a settings dictionary.
    for k,v in locals().items():
        if k not in ['settings']:
            settings[k] = v
        
    return settings


def create_directories(settings):
    for k in 'tracking_dir','output_dir','input_file','new_input_file':
        globals()[k] = settings[k]
    if not os.path.exists(tracking_dir):
        os.mkdir(tracking_dir)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    if not 'windows' in platform.system().lower():
        if not os.path.exists(new_input_file):
            os.symlink(os.path.relpath(input_file,output_dir),new_input_file)
    return

ModuleNotFoundError: No module named 'cv2'

# Locate input/output and allocate CPU's

`tracking_dir` sets the top-level output directory. The output of tracking each video will go in `tracking_dir`, in a subdirectory named after the input video file.  

`n_threads` controls the number of tracking tasks to execute simulataneously. `n_threads = None` defaults to the number of CPU's on the machine running the notebook. **This feature doesn't work in Windows, if you're using Windows

`input_files` set the list of video files to perform tracking on.

In [2]:
tracking_dir = '../tracking/full_20-07-14'
# tracking_dir = 'test'

def create_settings(input_file,tracking_dir=tracking_dir,settings=default_settings):
    return create_settings_(input_file,tracking_dir,settings)

#-----------------------------

n_threads = 6 # None
# Multi-threading doesn't work in Windows. If the OS is Windows, set n_thread to 1.
if 'windows' in platform.system().lower():
    n_threads = 1

#-----------------------------

# input_files = ['../raw_videos/dropbox/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000.avi']
# input_files = ['../raw_videos/dropbox/SF_Sat_14dpf_GroupB_n2a_2020-06-13-115534-0000.avi']
# input_files = ['../raw_videos/google-drive/SF_Sat_7dpf_GroupA_n2a_CORRUPT.avi']
# input_files = ['../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000.avi']
# input_files = ['../raw_videos/google-drive/Pa_Fri_7dpf_GroupB_n2b_2020-06-05-114635-0000.avi']
# input_files = ['../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupB_n2_2020-06-05-124632-0000_CORRUPT-repaired.avi']
# input_files = ['../raw_videos/google-drive/Pa_Fri_7dpf_GroupD_n5_2020-06-05-111448-0000.avi']

input_files = sorted(glob('../raw_videos/google-drive/Pa_Fri_7dpf_*.avi'))


# Filter out corrupt videos. Replace them with repaired video if available.
def swap_repaired_version(f):
    d,fn = os.path.split(f)
    fn,ext = os.path.splitext(fn)
    f2   = os.path.join(d,'repaired',fn+'-repaired'+ext)
    return f2 if os.path.exists(f2) else f

input_files = [ swap_repaired_version(f) for f in input_files ]


# Filter out videos that already have a tracking output in the current tracking_dir.
# input_files = [ f for f in input_files if not os.path.exists(create_settings(f)['trial_file']) ]


# input_files = input_files[1:2]
display(input_files)

['../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000.avi',
 '../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000.avi',
 '../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000.avi',
 '../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupB_n2_2020-06-05-124632-0000_CORRUPT-repaired.avi',
 '../raw_videos/google-drive/Pa_Fri_7dpf_GroupB_n2b_2020-06-05-114635-0000.avi',
 '../raw_videos/google-drive/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000.avi',
 '../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupC_n2_2020-06-05-132924-0000_CORRUPT-repaired.avi',
 '../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupC_n2b_2020-06-05-122929-0000_CORRUPT-repaired.avi',
 '../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupC_n5_CORRUPT-repaired.avi',
 '../raw_videos/google-drive/Pa_Fri_7dpf_GroupD_n2_2020-06-05-141433-0000.avi',
 '../raw_videos/google-drive/Pa_Fri_7dpf_GroupD_n2b_2020-06-05-131438-0000.avi',
 '../raw_videos/google-drive/Pa_Fri

# Locate the tanks

In [3]:
import cvt
reload(cvt.utils)
from cvt.utils import *
reload(cvt.TrAQ.Tank)
from cvt.TrAQ.Tank import Tank
reload(cvt.TrAQ.Tracker)
from cvt.TrAQ.Tracker import Tracker


for input_file in input_files:
    print(input_file)
    settings = create_settings(input_file)
    create_directories(settings)
    globals().update(settings)
    tank = Tank()
    tank.load_or_locate_and_save(tank_file,input_file)

../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000.avi
../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupB_n2_2020-06-05-124632-0000_CORRUPT-repaired.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupB_n2b_2020-06-05-114635-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000.avi
../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupC_n2_2020-06-05-132924-0000_CORRUPT-repaired.avi
../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupC_n2b_2020-06-05-122929-0000_CORRUPT-repaired.avi
../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupC_n5_CORRUPT-repaired.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupD_n2_2020-06-05-141433-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupD_n2b_2020-06-05-131438-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupD_n5_2020-06-05-111448-0000.avi


# Track

In [ ]:
import cvt
reload(cvt.utils)
from cvt.utils import *
reload(cvt.TrAQ.Tank)
from cvt.TrAQ.Tank import Tank
reload(cvt.TrAQ.Tracker)
from cvt.TrAQ.Tracker import Tracker


tweaks = dict( #t_start=815, t_end=855, 
#                ideal_area=50, ideal_aspect=3
             )


def track(input_file):
    
    sys.stdout.write(input_file+'\n')
    sys.stdout.flush()

    settings = create_settings(input_file)
    settings.update(tweaks)
    globals().update(settings)
    
    with open(log_file,'w') as f_log:
        
        f_log.write('Initializing...\n')
        tracker = Tracker( input_video=new_input_file, output_dir=output_dir, 
                           n_ind=Nfish, t_start=t_start, t_end=t_end, 
                           n_pixel_blur=n_pixel_blur, block_size=block_size, 
                           threshold_offset=thresh_offset, 
                           min_area=min_area, max_area=max_area, 
                           ideal_area=ideal_area, max_aspect=max_aspect, 
                           ideal_aspect=ideal_aspect, area_penalty=area_penalty, 
                           reversal_threshold=reversal_threshold, 
                           bkgSub_options=bkgSub_options, RGB=True, live_preview=online_viewer )

        tracker.save_settings()

        frames_dir = join(output_dir,'frames')

        try:
            tracker.set_frame(tracker.frame_start)
            tracker.init_live_preview()
            for i_frame in range(tracker.frame_start, tracker.frame_end+1):
                if i_frame%200==0:
                    i1,i2 = i_frame-tracker.frame_start, tracker.frame_end-tracker.frame_start
                    f_log.write(f'Tracking frame {i1}/{i2}...\n')
                    f_log.flush()
                
                if tracker.get_next_frame():
                    # Use b to output detailed information about the subset of the frames.
                    # Whenever b evaluates to True, images are saved in "[output_dir]/frames" 
                    # showing each step of the tracking process.
                    b = False # True # 260<tracker.frame_num<275 # 
                    if b:
                        if not os.path.exists(frames_dir):
                            os.mkdir(frames_dir)
    #                     cv2.imwrite(join(frames_dir,f'{tracker.frame_num}-1_raw.png'),tracker.frame)
                    tracker.subtract_background()
                    tracker.mask_tank()
                    tracker.detect_contours()
                    tracker.connect_frames()
                    if b:
#                         cv2.imwrite(join(frames_dir,f'{tracker.frame_num}-2_masked.png'),tracker.frame)
                        video_output_options.update(directors=False)
                        tracker.draw(**video_output_options)
                        cv2.imwrite(join(frames_dir,f'{tracker.frame_num}-3_contours.png'),tracker.frame)
                        video_output_options.update(directors=True)
                    tracker.draw(**video_output_options)
                    if b:
                        cv2.imwrite(join(frames_dir,f'{tracker.frame_num}-4_directors.png'),tracker.frame)
                    tracker.write_frame()
                    if not tracker.post_frame(delay=1):
                        break
#                     if i_frame%10==0:
                    tracker.print_current_frame()
            tracker.release()
            
        except:
            tracker.release()
            f_log.write('\nFailed\n')
            map(f_log.write,sys.exc_info())
            %tb
    
        f_log.write('Saving...\n')
        tracker.save_trial()
        
    return


# display(input_files)
n_threads = 6

if n_threads==1:

    for input_file in input_files:
        tracker = track(input_file)
    
else:
    
    pool = multiprocessing.Pool(n_threads)
    pool.map(track,input_files)


../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000.avi
../raw_videos/google-drive/repaired/Pa_Fri_7dpf_GroupB_n2_2020-06-05-124632-0000_CORRUPT-repaired.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupB_n2b_2020-06-05-114635-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000.avi
../raw_videos/google-drive/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000.avi


In [ ]:
# cv2.destroyAllWindows()

# Load output and plot trajectories

In [ ]:
display(input_files)

In [ ]:
for input_file in input_files[:1]:
    
    print(input_file)
    settings = create_settings(input_file)
#     settings.update(tweaks)
    globals().update(settings)
    globals().update(load_pik(trial_file))
    display(trial_file)
    
    # Plot trajectories.
    plt.figure(figsize=(8,)*2)
    circle = plt.Circle( (tank.x_px,tank.y_px), tank.r_px,
                         facecolor='None', edgecolor='k', lw=0.5 )
    plt.gca().add_patch(circle)
    for fish in range(data.shape[1]):
        x,y,theta,area = data[:,fish].T
#         plt.plot(x,y,marker='o',ms=2,mew=0.5,mfc='None',lw=0,label=str(fish))
        plt.plot(x,y,lw=1,label=str(fish))
    plt.axis('equal')
    plt.gca().yaxis.set_inverted(True)
    plt.legend()
    plt.show()


In [ ]:
np.sum(np.isnan(data))/data.size

In [ ]:
print(data.shape,data.size,np.sum(np.isnan(data)))